In [13]:

from typing import Any, Dict

import tensorflow as tf
import keras
from keras import backend as K
from keras.constraints import max_norm
from keras.layers import (
    Activation,
    Add,
    AveragePooling2D,
    AvgPool2D,
    BatchNormalization,
    Concatenate,
    Conv2D,
    Dense,
    DepthwiseConv2D,
    Dropout,
    Flatten,
    Input,
    Lambda,
    LayerNormalization,
    MaxPooling2D,
    Permute,
)
from keras.models import Model, Sequential
from scikeras.wrappers import KerasClassifier

from moabb.pipelines.utils_deep_model import EEGNet, EEGNet_TC

from pickle import load

import keras
# from braindecode import EEGClassifier
# from braindecode.models import EEGInception
# from scikeras.wrappers import KerasClassifier
# from sklearn.pipeline import Pipeline, make_pipeline
# from skorch.callbacks import EarlyStopping, EpochScoring
# from skorch.dataset import ValidSplit

from moabb import set_log_level
from moabb.pipelines.features import StandardScaler_Epoch
from moabb.utils import setup_seed

# Datasets
from custom_datasets.fatigue_mi import FatigueMI
from custom_datasets.norm_cho import NormCho2017
from moabb.datasets import Cho2017, Zhou2016, Weibo2014

# Moabb evaluation methods
from moabb.evaluations import CrossSessionEvaluation, WithinSessionEvaluation

# Moabb paradigms
from moabb.paradigms import LeftRightImagery

# Scikitlearn imports
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

from moabb.pipelines.features import Resampler_Epoch, Convert_Epoch_Array, StandardScaler_Epoch


%load_ext autoreload
%autoreload 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# Set up reproducibility of Tensorflow
setup_seed(42)

In [15]:
# =================================================================================
# DeepConvNet
# =================================================================================
class KerasDeepConvNet(KerasClassifier):
    """Keras implementation of the Deep Convolutional Network as described in
    [1]_.

    This implementation is taken from code by the Army Research Laboratory (ARL)
    at https://github.com/vlawhern/arl-eegmodels

    We use the original parameter implemented on the paper.

    Note that this implementation has not been verified by the original
    authors.

    References
    ----------
    .. [1] Schirrmeister, R. T., Springenberg, J. T., Fiederer, L. D. J., Glasstetter, M., Eggensperger,
           K., Tangermann, M., ... & Ball, T. (2017). Deep learning with convolutional neural networks
           for EEG decoding and visualization. Human brain mapping, 38(11), 5391-5420.
           https://doi.org/10.1002/hbm.23730

    Notes
    -----
    .. versionadded:: 0.5.0
    """

    def __init__(
        self,
        loss,
        optimizer="Adam",
        epochs=1000,
        batch_size=64,
        verbose=0,
        random_state=None,
        validation_split=0.2,
        history_plot=False,
        path=None,
        **kwargs,
    ):
        super().__init__(**kwargs)

        self.loss = loss
        if optimizer == "Adam":
            optimizer = tf.keras.optimizers.Adam(learning_rate=0.0009)

        self.optimizer = optimizer
        self.epochs = epochs
        self.batch_size = batch_size
        self.verbose = verbose
        self.random_state = random_state
        self.validation_split = validation_split
        self.history_plot = history_plot
        self.path = path

    def _keras_build_fn(self, compile_kwargs: Dict[str, Any]):
        input_main = Input(shape=(self.X_shape_[1], self.X_shape_[2], 1))

        block1 = Conv2D(
            25,
            (1, 10),
            input_shape=(self.X_shape_[1], self.X_shape_[2], 1),
            kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        )(input_main)
        block1 = Conv2D(
            25,
            (self.X_shape_[1], 1),
            kernel_constraint=max_norm(2.0, axis=(0, 1, 2)),
        )(block1)
        block1 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block1)
        block1 = Activation("elu")(block1)
        block1 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block1)
        block1 = Dropout(0.5)(block1)

        block2 = Conv2D(50, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block1
        )
        block2 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block2)
        block2 = Activation("elu")(block2)
        block2 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block2)
        block2 = Dropout(0.5)(block2)

        block3 = Conv2D(100, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block2
        )
        block3 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block3)
        block3 = Activation("elu")(block3)
        block3 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block3)
        block3 = Dropout(0.5)(block3)

        block4 = Conv2D(200, (1, 10), kernel_constraint=max_norm(2.0, axis=(0, 1, 2)))(
            block3
        )
        block4 = BatchNormalization(epsilon=1e-05, momentum=0.9)(block4)
        block4 = Activation("elu")(block4)
        block4 = MaxPooling2D(pool_size=(1, 3), strides=(1, 3))(block4)
        block4 = Dropout(0.5)(block4)

        flatten = Flatten()(block4)

        dense = Dense(self.n_classes_, kernel_constraint=max_norm(0.5))(flatten)
        softmax = Activation("softmax")(dense)

        model = Model(inputs=input_main, outputs=softmax)
        
        model.compile(loss=compile_kwargs["loss"], optimizer=compile_kwargs["optimizer"])

        ## Pruning

        # prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

        # pruning_params = {
        #     'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(0.5, begin_step=0, frequency=100)
        # }

        # callbacks = [
        # tfmot.sparsity.keras.UpdatePruningStep()
        # ]

        # pruned_model = prune_low_magnitude(model, **pruning_params)

        # # Use smaller learning rate for fine-tuning
        # opt = keras.optimizers.Adam(learning_rate=1e-5)

        # pruned_model.compile(
        # loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        # optimizer=opt,
        # metrics=['accuracy'])

        return model
        

keras_deep_conv_net = KerasDeepConvNet(
    loss="sparse_categorical_crossentropy",
    optimizer="Adam",
    epochs=5,
    batch_size=64,
    verbose=1,
    random_state=None,
    validation_split=0.2,
    history_plot=False,
    path=None,
)
# keras_deep_conv_net.save("keras_deep_conv_net.h5")

In [17]:
# model_keras_pruned = keras.models.load_model("keras_deep_conv_net.h5")
model_keras_pruned = keras_deep_conv_net
# Now we need to instantiate a new SciKeras object since we only saved the Keras model
resampler_epoch = Resampler_Epoch(sfreq=250)
convert_epoch_array = Convert_Epoch_Array()
standardscaler_epoch = StandardScaler_Epoch()
Keras_DeepConvNet_Trained_Pruned = make_pipeline(
    resampler_epoch,
    convert_epoch_array,
    standardscaler_epoch,
    model_keras_pruned
)

Keras_DeepConvNet_Trained_Pruned

Pipeline(steps=[('resampler_epoch', Resampler_Epoch(sfreq=250)),
                ('convert_epoch_array', Convert_Epoch_Array()),
                ('standardscaler_epoch', StandardScaler_Epoch()),
                ('kerasdeepconvnet',
                 KerasDeepConvNet(build_fn=None, callbacks=None, class_weight=None, epochs=25, loss='sparse_categorical_crossentropy', metrics=None, model=None, optimizer=<keras.optimizers.adam.Adam object at 0x7f39bde75950>, run_eagerly=False, shuffle=True, validation_batch_size=None, verbose=1, warm_start=False))])

In [19]:
# # model_keras_pruned = keras.models.load_model("keras_deep_conv_net.h5")
# model_keras_pruned = keras_deep_conv_net
# # Now we need to instantiate a new SciKeras object since we only saved the Keras model
# Keras_DeepConvNet_Trained_Pruned = KerasClassifier(build_fn = model_keras_pruned._keras_build_fn)

# Restrict this example only on the first two subject of BNCI2014_001
fat_dataset = FatigueMI()
norm_cho_dataset = NormCho2017()
cho_dataset = Cho2017()

fat_dataset.subject_list = fat_dataset.subject_list[:1]
norm_cho_dataset.subject_list = norm_cho_dataset.subject_list[:32]
cho_dataset.subject_list = cho_dataset.subject_list[:32]

datasets = [
    fat_dataset,
    # norm_cho_dataset,
    # cho_dataset
]

evaluation_dict = {
    "WithinSession": WithinSessionEvaluation
}
paradim_dict = {
    "LeftRightImagery": LeftRightImagery
}
pipeline_dict = {
    "Keras_DeepConvNet_Trained_Pruned": Pipeline(
        [
            # ("StandardScaler_Epoch", StandardScaler_Epoch),
            ("Keras_DeepConvNet_Trained", Keras_DeepConvNet_Trained_Pruned),
        ]
    ),
}

results_dict = {}

for model_text in pipeline_dict.keys():
    for evaluation_text in ["WithinSession"]: #, "CrossSession", "CrossSubject"]:
        for paradim_text in ["LeftRightImagery"]: #, "FilterBankMotorImagery"]:
            evaluation = evaluation_dict[evaluation_text](
                paradigm=paradim_dict[paradim_text](), datasets=datasets, overwrite=True, return_epochs=True,
            )
            results = evaluation.process({ model_text: pipeline_dict[model_text] })
            results_dict[model_text + "_" + evaluation_text + "_" + paradim_text] = results



FatigueMI-WithinSession:   0%|          | 0/1 [00:00<?, ?it/s]

Adding metadata with 3 columns
Adding metadata with 3 columns
108 matching events found
No baseline correction applied
No hdf5_path provided, models will not be saved.


/home/araz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/moabb/paradigms/base.py:354: RuntimeWarning: Concatenation of Annotations within Epochs is not supported yet. All annotations will be dropped.
  X = mne.concatenate_epochs(X)
/home/araz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/moabb/pipelines/features.py:163: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return X.get_data()


Epoch 1/25


2024-02-16 14:09:42.327927: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [68,1]
	 [[{{node Placeholder/_11}}]]
2024-02-16 14:09:42.328817: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [68,1]
	 [[{{node Placeholder/_11}}]]
2024-02-16 14:09:43.555128: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 62976000 exceeds 10% of free system memory.


...

2024-02-16 14:12:47.739050: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [18,20,501]
	 [[{{node Placeholder/_10}}]]
2024-02-16 14:12:47.745856: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_11' with dtype float and shape [18,1]
	 [[{{node Placeholder/_11}}]]


2/2 [==============================] - 2s 908ms/step - loss: 1.3019 - val_loss: 0.6999
Epoch 2/25
2/2 [==============================] - 0s 37ms/step - loss: 0.8159 - val_loss: 0.7141
Epoch 3/25
2/2 [==============================] - 0s 35ms/step - loss: 0.7946 - val_loss: 0.7371
Epoch 4/25
2/2 [==============================] - 0s 28ms/step - loss: 0.8319 - val_loss: 0.7571
Epoch 5/25
2/2 [==============================] - 0s 26ms/step - loss: 0.8588 - val_loss: 0.7795
Epoch 6/25
2/2 [==============================] - 0s 26ms/step - loss: 0.9026 - val_loss: 0.7939
Epoch 7/25
2/2 [==============================] - 0s 28ms/step - loss: 0.6896 - val_loss: 0.7904
Epoch 8/25
2/2 [==============================] - 0s 27ms/step - loss: 0.8281 - val_loss: 0.7835
Epoch 9/25
2/2 [==============================] - 0s 34ms/step - loss: 0.7553 - val_loss: 0.7711
Epoch 10/25
2/2 [==============================] - 0s 28ms/step - loss: 0.7645 - val_loss: 0.7537
Epoch 11/25
2/2 [======================

/home/araz/anaconda3/envs/moabb_model_optimization/lib/python3.11/site-packages/moabb/pipelines/features.py:163: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  return X.get_data()
2024-02-16 14:12:56.564394: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,501]
	 [[{{node Placeholder/_10}}]]
2024-02-16 14:12:56.580523: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_10' with dtype float and shape [21,20,501]
	 [[{{node Placeholder/_10}}]]


1/1 [==============================] - 1s 1s/step


FatigueMI-WithinSession: 100%|██████████| 1/1 [03:16<00:00, 196.70s/it]


In [20]:
from rich.pretty import pprint as rpprint

rpprint(
    results_dict['Keras_DeepConvNet_Trained_Pruned_WithinSession_LeftRightImagery'],
    expand_all=True
)

│     score       time  samples subject session  channels  n_sessions  \
0  0.497851  39.150051    108.0       1       0        20           1   
│    
│    dataset                          pipeline  
0  FatigueMI  Keras_DeepConvNet_Trained_Pruned